In [4]:
import openmeteo_requests
import requests_cache
import pandas as pd

from retry_requests import retry
from utils import dataframe_info, racine_projet

In [2]:
racine_projet()

'/Users/kfranceschi/GitHub/projet-pompiers'

### Code API pour l'import importé de  : https://open-meteo.com/en/docs

In [4]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.5085,
	"longitude": -0.1257,
	"start_date": "2009-01-01",
	"end_date": "2024-05-30",
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "weather_code", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
#hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
#hourly_wind_direction_10m = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
#hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["weather_code"] = hourly_weather_code
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
#hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)

Coordinates 51.49384689331055°N -0.16302490234375°E
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


### Infos sur le dataframe créé et export sous format csv

In [5]:
dataframe_info(hourly_dataframe)

,Column,Non-Null Count,NaN Count,NaN Percentage,Dtype,Example Value
0,date,135096,0,0.0,"datetime64[ns, UTC]",2009-01-01 00:00:00+00:00
1,temperature_2m,135096,0,0.0,float32,-0.3
2,relative_humidity_2m,135096,0,0.0,float32,96.071747
3,rain,135096,0,0.0,float32,0.0
4,weather_code,135096,0,0.0,float32,2.0
5,wind_speed_10m,135096,0,0.0,float32,6.489992
6,wind_gusts_10m,135096,0,0.0,float32,12.959999


In [7]:
hourly_dataframe.to_csv(racine_projet()+'/data/external/weather.csv', index=False)